In [1]:
# importing necessary libraries
import numpy as np
import pandas as pd

In [2]:
# read the train and test sets
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
train.head()




,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
# filling location and keyword with unknown and none
train['location'].fillna('Unknown', inplace=True)
test['location'].fillna('Unknown', inplace=True)
train['keyword'].fillna('None', inplace=True)
test['keyword'].fillna('None', inplace=True)


In [4]:
# separate targe and features

X = train[['text', 'location', 'keyword']]
y = train['target']



In [5]:
#train validaztion split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)



In [6]:
# combine all features as a single string
X_train['all'] = X_train['text'] + ' ' + X_train['location'] + ' ' + X_train['keyword']
X_val['all'] = X_val['text'] + ' ' + X_val['location'] + ' ' + X_val['keyword']
test['all'] = test['text'] + ' ' + test['location'] + ' ' + test['keyword']



<ipython-input-6-22b434eb261c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['all'] = X_train['text'] + ' ' + X_train['location'] + ' ' + X_train['keyword']
<ipython-input-6-22b434eb261c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val['all'] = X_val['text'] + ' ' + X_val['location'] + ' ' + X_val['keyword']


In [7]:
# model with distilledbert
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
from transformers import TFDistilBertModel
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# load the tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# encode the text
X_train_encoded = tokenizer(X_train['all'].tolist(), padding=True, truncation=True, max_length=512)
X_val_encoded = tokenizer(X_val['all'].tolist(), padding=True, truncation=True, max_length=512)
test_encoded = tokenizer(test['all'].tolist(), padding=True, truncation=True, max_length=512)


c:\Users\lenovo\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lenovo\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [8]:
# create the model
def create_model():
    bert = TFDistilBertModel.from_pretrained('distilbert-base-uncased')
    input_ids = Input(shape=(512,), name='input_ids', dtype='int32')
    attention_mask = Input(shape=(512,), name='attention_mask', dtype='int32')
    bert_input = {'input_ids': input_ids, 'attention_mask': attention_mask}
    bert_output = bert(bert_input)['last_hidden_state']
    output = Dense(1, activation='sigmoid')(bert_output[:,0,:])
    model = Model(inputs=[input_ids, attention_mask], outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [9]:
# create the model
model = create_model()
model.summary()


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

In [12]:
# train the model
import tensorflow as tf

# Convert input lists to tensors
X_train_input_ids = tf.convert_to_tensor(X_train_encoded['input_ids'])
X_train_attention_mask = tf.convert_to_tensor(X_train_encoded['attention_mask'])
X_val_input_ids = tf.convert_to_tensor(X_val_encoded['input_ids'])
X_val_attention_mask = tf.convert_to_tensor(X_val_encoded['attention_mask'])

# Train the model with the converted tensors
model.fit([X_train_input_ids, X_train_attention_mask], y_train, validation_data=([X_val_input_ids, X_val_attention_mask], y_val), epochs=3, batch_size=8)





Epoch 1/3
762/762 [==============================] - 1386s 2s/step - loss: 0.7460 - accuracy: 0.5420 - val_loss: 0.6828 - val_accuracy: 0.5739
Epoch 2/3
762/762 [==============================] - 1181s 2s/step - loss: 0.6922 - accuracy: 0.5494 - val_loss: 0.6822 - val_accuracy: 0.5739
Epoch 3/3
762/762 [==============================] - 1171s 2s/step - loss: 0.6908 - accuracy: 0.5555 - val_loss: 0.6865 - val_accuracy: 0.5739


In [13]:
# predict on the test set	
test_input_ids = tf.convert_to_tensor(test_encoded['input_ids'])
test_attention_mask = tf.convert_to_tensor(test_encoded['attention_mask'])
y_pred = model.predict([test_input_ids, test_attention_mask])

# create a submission file
submission = pd.read_csv('data/sample_submission.csv')
submission['target'] = np.round(y_pred).astype(int)
submission.to_csv('submission.csv', index=False)